# Transfert learning with VVG16

In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
from PIL import Image

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [24]:
! ls ../raw_data/

all_data_info.csv train             train_1_resized
test              train_1


In [25]:
def load_data(loading_method):
    
    """
    Create the datasets X_train, y_train, X_val, y_val, X_test, y_test, num_classes
    
    Each number of classe is an artist
    
    """
    
    if loading_method == 'colab':
        data_path = '/content/drive/My Drive/Deep_learning_data/flowers'
    elif loading_method == 'direct':
        data_path = 'flowers/'
    
    
    #classes = get_classes #{'daisy':0, 'dandelion':1, 'rose':2}
    
    train_directory = '../raw_data/train/'
    test_directory = '../raw_data/test/'
    
    
    from tensorflow.keras.preprocessing import image_dataset_from_directory
    
    train_dataset = image_dataset_from_directory(train_directory,
    labels="inferred",
    label_mode="int",
    #class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False)
    
    test_dataset = image_dataset_from_directory(test_directory,
    labels="inferred",
    label_mode="int",
    #class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False)
    
    return train_dataset
    
    #return X_train, y_train, X_val, y_val, X_test, y_test, num_classes

In [26]:
train_dataset = load_data('direct')

Found 17 files belonging to 3 classes.
Found 9 files belonging to 3 classes.


In [28]:
train_dataset[1]
#X_train, y_train, X_val, y_val, X_test, y_test, num_classes = load_data('direct')

TypeError: 'BatchDataset' object is not subscriptable

In [14]:
from tensorflow.keras.applications.vgg16 import VGG16

def load_model():
    model = VGG16(weights="imagenet", include_top=False, input_shape=X_train[0].shape)
    return model

In [15]:
model = load_model()
model.summary()

TypeError: object of type 'int' has no len()

In [ ]:
def set_nontrainable_layers(model):
    # Set the first layers to be untrainable
    model.trainable = False
    
    return model

def add_last_layers(model):
    number_of_classes = len(num_classes)
    
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(number_of_classes, activation='softmax')
    
    
    model = models.Sequential([
        model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    
    return model

In [ ]:
model = add_last_layers(model)

In [ ]:
def compile_model(model):
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

In [ ]:
def build_model():
    
    model = load_model()
    model = add_last_layers(model)
    model = compile_model(model)
    
    return model

model = build_model()

# Run the model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_acc', mode='max', patience=5, verbose=1, restore_best_weights=True)

history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs=50, 
                    batch_size=16, 
                    callbacks=[es])

In [ ]:
# Plot history
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.show()

In [ ]:
model.evaluate(x_test, y_test)

print(f'Chance level: {1./num_classes*100:.4f}%')